# 方法設計

方法構成了Julia 程式當中的絕大多數的部份，也是 Julia 在多重分派中重要的角色。我們在設計方法的時候需要考慮的是廣義化，一個好的方法被設計出來，希望他不只能夠應用於少數情況，我們會希望他可以適用於更多的情境。我們希望一個方法可以被設計的夠俱備一般性，如此才能適用更多情境。

In [1]:
function plus(x, y)
    return x + y
end

plus (generic function with 1 method)

以上是一個乍看非常簡單，但是異常糟糕的方法實作。在 API 的部份，並沒有去界定 `x` 及 `y` 兩個參數的範圍，如此就允許接收各式各樣型別的物件，這樣很容易導致錯誤。如此一來，使用者也無從得知什麼時候該使用這個方法，便會以為任何型別的物件都可以進行運算，但是事實並非如此。

## API 設計 - 依目的設計

首先，我們需要界定參數型別的範圍。一般來說，需要依據設計者的設計目的而制定。例如：

In [2]:
function plus(x::Vector, y::Vector)
    return x + y
end

plus (generic function with 2 methods)

以上的實作多了一些資訊，但仍舊不足。`plus` 牽涉到的是 `Vector` 當中值的運算，但這邊並沒有去界定值的型別，我們可以利用參數化來處理，最廣義的形式是這個樣子。

In [3]:
function plus(x::Vector{T}, y::Vector{S}) where {T,S}
    return x + y
end

plus (generic function with 2 methods)

我們用參數化來處理，但是並沒有增加任何的限制，所以我們可以在後面加上型別的限制。

In [4]:
function plus(x::Vector{T}, y::Vector{S}) where {T<:Number,S<:Number}
    return x + y
end

plus (generic function with 3 methods)

如此我們完成了對 API 的設計，這是一個可以對一維陣列做加法運算的方法，陣列當中的值要是 `Number` 的子型別。

## 實作設計 - 依所支援的方法設計

接下來，我們需要檢查方法所接收的參數與內部實作之間的關係。參數有其型別，而我們也在 API 上定義的了型別，型別有各自的方法。我們需要確認我們所設計的 `plus` 中所使用的方法都是有支援的。

使用到的 API：

* `+`

在我們的範例中，`plus` 使用到的方法只有一個，就是 `+`。`+` 會直接作用在 `x` 及 `y` 身上，我們需要確認他們的型別 `Vector{T}` 及 `Vector{S}` 是有支援 `+` 的。這件事我們一般可以由官方文件查詢得到，或是我們可以直接執行並測試我們的實作得到驗證。

In [5]:
plus([1,2,3], [4,5,6])

3-element Array{Int64,1}:
 5
 7
 9

如果使用的是自己設計的方法或是套件，請再次確認被使用到的 API 是有實作的。